# Enterprise Business Performance Analytics

This notebook contains the end-to-end analysis for an e-commerce dataset.
The focus is on building clean data, validating it, and creating KPIs
used for finance and operations reporting.

## 1. Project Objective

The purpose of this analysis is to:
- clean and prepare raw transactional data
- validate data quality before reporting
- create a consistent KPI framework
- analyze performance trends over time

The output is intended to support operational and strategic decision-making.


## 2. Data Source and Context

The dataset contains order, item, and payment level data from an e-commerce
platform. It is used here to simulate common finance and operations
analytics use cases such as revenue tracking and delivery performance.


## 3. Data Ingestion (Raw Layer)

Raw CSV files were loaded into a local SQLite database.
This step centralizes the data and enables consistent SQL-based analysis
across all tables.


In [ ]:
!pip install pandas sqlalchemy


In [4]:
import os
import pandas as pd
from sqlalchemy import create_engine

# Your exact raw data folder
RAW_DIR = r"C:\Users\vedas\Downloads\enterprise-analytics-platform\data\raw"
DB_PATH = os.path.join(RAW_DIR, "enterprise_analytics.db")

engine = create_engine(f"sqlite:///{DB_PATH}")

files = {
    "olist_orders": "olist_orders_dataset.csv",
    "olist_order_items": "olist_order_items_dataset.csv",
    "olist_order_payments": "olist_order_payments_dataset.csv",
    "olist_customers": "olist_customers_dataset.csv",
    "olist_products": "olist_products_dataset.csv",
    "olist_sellers": "olist_sellers_dataset.csv",
    "olist_order_reviews": "olist_order_reviews_dataset.csv",
    "product_category_translation": "product_category_name_translation.csv",
}

loaded = []

for table, filename in files.items():
    path = os.path.join(RAW_DIR, filename)

    if not os.path.exists(path):
        print(f"SKIP missing file: {filename}")
        continue

    df = pd.read_csv(path)
    df.columns = [c.strip().lower() for c in df.columns]
    df.to_sql(table, engine, if_exists="replace", index=False)
    loaded.append((table, len(df)))

print("\n✅ Database created and tables loaded:")
for t, n in loaded:
    print(f" - {t}: {n:,} rows")

print(f"\n✅ Database file created here: {DB_PATH}")



✅ Database created and tables loaded:
 - olist_orders: 99,441 rows
 - olist_order_items: 112,650 rows
 - olist_order_payments: 103,886 rows
 - olist_customers: 99,441 rows
 - olist_products: 32,951 rows
 - olist_sellers: 3,095 rows
 - olist_order_reviews: 99,224 rows
 - product_category_translation: 71 rows

✅ Database file created here: C:\Users\vedas\Downloads\enterprise-analytics-platform\data\raw\enterprise_analytics.db


In [5]:
import sqlite3

con = sqlite3.connect(r"C:\Users\vedas\Downloads\enterprise-analytics-platform\data\raw\enterprise_analytics.db")
cur = con.cursor()

cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = cur.fetchall()
tables



[('olist_customers',),
 ('olist_order_items',),
 ('olist_order_payments',),
 ('olist_order_reviews',),
 ('olist_orders',),
 ('olist_products',),
 ('olist_sellers',),
 ('product_category_translation',)]

In [6]:
cur.execute("""
SELECT COUNT(*) AS total_rows, COUNT(DISTINCT order_id) AS distinct_order_id
FROM olist_orders;
""")
cur.fetchall()

[(99441, 99441)]

In [7]:
con.close()
print("✅ Verification done")


✅ Verification done


## 4. Staging Layer – Orders

Order data was cleaned and standardized in a staging table.
Date fields were normalized and records missing critical keys
were removed to ensure data reliability.



In [8]:
import sqlite3

# Database path
db_path = r"C:\Users\vedas\Downloads\enterprise-analytics-platform\data\raw\enterprise_analytics.db"

# SQL file path
sql_file_path = r"C:\Users\vedas\Downloads\enterprise-analytics-platform\sql\01_staging_orders.sql"

# Connect to database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Read SQL file
with open(sql_file_path, "r", encoding="utf-8") as file:
    sql_script = file.read()

# Execute SQL
cursor.executescript(sql_script)

# Save changes
conn.commit()
conn.close()

print("✅ stg_orders table created successfully")


✅ stg_orders table created successfully


In [9]:
import sqlite3

conn = sqlite3.connect(r"C:\Users\vedas\Downloads\enterprise-analytics-platform\data\raw\enterprise_analytics.db")
cursor = conn.cursor()

cursor.execute("SELECT COUNT(*) FROM stg_orders;")
cursor.fetchall()


[(99441,)]

In [10]:
cursor.execute("SELECT * FROM stg_orders LIMIT 5;")
cursor.fetchall()

[('e481f51cbdc54678b7cc49136f2d6af7',
  '9ef432eb6251297304e76186b10a928d',
  'delivered',
  '2017-10-02',
  '2017-10-02',
  '2017-10-04',
  '2017-10-10',
  '2017-10-18'),
 ('53cdb2fc8bc7dce0b6741e2150273451',
  'b0830fb4747a6c6d20dea0b8c802d7ef',
  'delivered',
  '2018-07-24',
  '2018-07-26',
  '2018-07-26',
  '2018-08-07',
  '2018-08-13'),
 ('47770eb9100c2d0c44946d9cf07ec65d',
  '41ce2a54c0b03bf3443c3d931a367089',
  'delivered',
  '2018-08-08',
  '2018-08-08',
  '2018-08-08',
  '2018-08-17',
  '2018-09-04'),
 ('949d5b44dbf5de918fe9c16f97b45f8a',
  'f88197465ea7920adcdbec7375364d82',
  'delivered',
  '2017-11-18',
  '2017-11-18',
  '2017-11-22',
  '2017-12-02',
  '2017-12-15'),
 ('ad21c59c0840e6cb83a9ceb5573f8159',
  '8ab97904e6daea8866dbdbc4fb7aad2c',
  'delivered',
  '2018-02-13',
  '2018-02-13',
  '2018-02-14',
  '2018-02-16',
  '2018-02-26')]

In [11]:
conn.close()
print("✅ Verification done")


✅ Verification done


## 5. Staging Layer – Order Items

Order item data was staged to prepare pricing and freight values
for revenue and cost analysis. Only valid records with required
keys were retained.


In [12]:
import sqlite3

db_path = r"C:\Users\vedas\Downloads\enterprise-analytics-platform\data\raw\enterprise_analytics.db"
sql_file_path = r"C:\Users\vedas\Downloads\enterprise-analytics-platform\sql\02_staging_order_items.sql"

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

with open(sql_file_path, "r", encoding="utf-8") as file:
    sql_script = file.read()

cursor.executescript(sql_script)
conn.commit()
conn.close()

print("✅ stg_order_items table created successfully")


✅ stg_order_items table created successfully


In [13]:
import sqlite3

conn = sqlite3.connect(r"C:\Users\vedas\Downloads\enterprise-analytics-platform\data\raw\enterprise_analytics.db")
cursor = conn.cursor()

cursor.execute("SELECT COUNT(*) FROM stg_order_items;")
cursor.fetchall()


[(112650,)]

In [14]:
cursor.execute("SELECT * FROM stg_order_items LIMIT 5;")
cursor.fetchall()


[('00010242fe8c5a6d1ba2dd792cb16214',
  1,
  '4244733e06e7ecb4970a6e2683c13e61',
  '48436dade18ac8b2bce089ec2a041202',
  '2017-09-19',
  58.9,
  13.29),
 ('00018f77f2f0320c557190d7a144bdd3',
  1,
  'e5f2d52b802189ee658865ca93d83a8f',
  'dd7ddc04e1b6c2c614352b383efe2d36',
  '2017-05-03',
  239.9,
  19.93),
 ('000229ec398224ef6ca0657da4fc703e',
  1,
  'c777355d18b72b67abbeef9df44fd0fd',
  '5b51032eddd242adc84c38acab88f23d',
  '2018-01-18',
  199.0,
  17.87),
 ('00024acbcdf0a6daa1e931b038114c75',
  1,
  '7634da152a4610f1595efa32f14722fc',
  '9d7a1d34a5052409006425275ba1c2b4',
  '2018-08-15',
  12.99,
  12.79),
 ('00042b26cf59d7ce69dfabb4e55b4fd9',
  1,
  'ac6c3623068f30de03045865e4e10089',
  'df560393f3a51e74553ab94004ba5c87',
  '2017-02-13',
  199.9,
  18.14)]

In [15]:
cursor.execute("""
SELECT
  COUNT(*) AS total_rows,
  COUNT(DISTINCT order_id || '-' || order_item_id) AS distinct_keys
FROM stg_order_items;
""")
cursor.fetchall()


[(112650, 112650)]

In [16]:
conn.close()
print("✅ Verification done")


✅ Verification done


## 6. Staging Layer – Payments

Payment data was staged to capture how much was paid per order
and the associated payment attributes. This table is used
as the source of truth for revenue calculations.


In [17]:
import sqlite3

db_path = r"C:\Users\vedas\Downloads\enterprise-analytics-platform\data\raw\enterprise_analytics.db"
sql_file_path = r"C:\Users\vedas\Downloads\enterprise-analytics-platform\sql\03_staging_payments.sql"

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

with open(sql_file_path, "r", encoding="utf-8") as file:
    sql_script = file.read()

cursor.executescript(sql_script)
conn.commit()
conn.close()

print("✅ stg_payments table created successfully")


✅ stg_payments table created successfully


In [18]:
import sqlite3

conn = sqlite3.connect(r"C:\Users\vedas\Downloads\enterprise-analytics-platform\data\raw\enterprise_analytics.db")
cursor = conn.cursor()

cursor.execute("SELECT COUNT(*) FROM stg_payments;")
cursor.fetchall()


[(103886,)]

In [19]:
cursor.execute("SELECT * FROM stg_payments LIMIT 5;")
cursor.fetchall()


[('b81ef226f3fe1789b1e8b2acac839d17', 1, 'credit_card', 8, 99.33),
 ('a9810da82917af2d9aefd1278f1dcfa0', 1, 'credit_card', 1, 24.39),
 ('25e8ea4e93396b6fa0d3dd708e76c1bd', 1, 'credit_card', 1, 65.71),
 ('ba78997921bbcdc1373bb41e913ab953', 1, 'credit_card', 8, 107.78),
 ('42fdf880ba16b47b59251dd489d4441a', 1, 'credit_card', 2, 128.45)]

In [20]:
cursor.execute("""
SELECT order_id, ROUND(SUM(payment_value), 2) AS total_paid
FROM stg_payments
GROUP BY order_id
ORDER BY total_paid DESC
LIMIT 5;
""")
cursor.fetchall()


[('03caa2c082116e1d31e67e9ae3700499', 13664.08),
 ('736e1922ae60d0d6a89247b851902527', 7274.88),
 ('0812eb902a67711a1cb742b3cdaa65ae', 6929.31),
 ('fefacc66af859508bf1a7934eab1e97f', 6922.21),
 ('f5136e38d1a14a4dbd87dff67da82701', 6726.66)]

In [21]:
conn.close()
print("✅ Verification done")


✅ Verification done


## 7. Analytics Base Table

Orders, order items, and payment data were combined into a single
fact table. This table serves as the primary source for all KPIs
and downstream analysis.


In [22]:
import sqlite3

db_path = r"C:\Users\vedas\Downloads\enterprise-analytics-platform\data\raw\enterprise_analytics.db"
sql_file_path = r"C:\Users\vedas\Downloads\enterprise-analytics-platform\sql\04_create_fact_order_lines.sql"

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

with open(sql_file_path, "r", encoding="utf-8") as f:
    sql_script = f.read()

cursor.executescript(sql_script)
conn.commit()
conn.close()

print("✅ fct_order_lines created successfully")


✅ fct_order_lines created successfully


In [23]:
import sqlite3

conn = sqlite3.connect(r"C:\Users\vedas\Downloads\enterprise-analytics-platform\data\raw\enterprise_analytics.db")
cursor = conn.cursor()

cursor.execute("SELECT COUNT(*) FROM fct_order_lines;")
cursor.fetchall()


[(112650,)]

In [24]:
cursor.execute("SELECT * FROM fct_order_lines LIMIT 5;")
cursor.fetchall()


[('e481f51cbdc54678b7cc49136f2d6af7',
  '9ef432eb6251297304e76186b10a928d',
  'delivered',
  '2017-10-02',
  '2017-10-04',
  '2017-10-10',
  '2017-10-18',
  1,
  '87285b34884572647811a353c7ac498a',
  '3504c0cb71d7fa48d967e0e4c94d59d9',
  '2017-10-06',
  29.99,
  8.72,
  38.71,
  1,
  8,
  1),
 ('53cdb2fc8bc7dce0b6741e2150273451',
  'b0830fb4747a6c6d20dea0b8c802d7ef',
  'delivered',
  '2018-07-24',
  '2018-07-26',
  '2018-08-07',
  '2018-08-13',
  1,
  '595fac2a385ac33a80bd5114aec74eb8',
  '289cdb325fb7e7f891c38608bf9e0962',
  '2018-07-30',
  118.7,
  22.76,
  141.46,
  1,
  14,
  1),
 ('47770eb9100c2d0c44946d9cf07ec65d',
  '41ce2a54c0b03bf3443c3d931a367089',
  'delivered',
  '2018-08-08',
  '2018-08-08',
  '2018-08-17',
  '2018-09-04',
  1,
  'aa4383b373c6aca5d8797843e5594415',
  '4869f7a5dfa277a7dca6462dcf3b52b2',
  '2018-08-13',
  159.9,
  19.22,
  179.12,
  1,
  9,
  1),
 ('949d5b44dbf5de918fe9c16f97b45f8a',
  'f88197465ea7920adcdbec7375364d82',
  'delivered',
  '2017-11-18',
  '201

In [25]:
cursor.execute("""
SELECT SUM(CASE WHEN order_id IS NULL THEN 1 ELSE 0 END) AS null_order_id
FROM fct_order_lines;
""")
cursor.fetchall()


[(0,)]

In [26]:
conn.close()
print("✅ Verification done")


✅ Verification done


# Data Quality Report (Governance + QA)

## 8. Data Quality Checks

Before calculating KPIs, data quality checks were performed to verify:
- no missing primary keys
- no duplicate order-item combinations
- valid price and payment values
- correct delivery date logic

All checks passed before proceeding.


In [30]:
import sqlite3
import pandas as pd

db_path = r"C:\Users\vedas\Downloads\enterprise-analytics-platform\data\raw\enterprise_analytics.db"
conn = sqlite3.connect(db_path)

print("✅ Connected to SQLite DB")


✅ Connected to SQLite DB


In [31]:
df = pd.read_sql_query("SELECT * FROM fct_order_lines;", conn)
df.shape

(112650, 17)

In [32]:
tests = []

def add_test(name, failed_count, total_rows):
    tests.append({
        "test_name": name,
        "failed_rows": int(failed_count),
        "total_rows": int(total_rows),
        "failed_pct": round((failed_count / total_rows) * 100, 4) if total_rows else 0.0,
        "status": "PASS" if failed_count == 0 else "FAIL"
    })

total = len(df)

# 1) Null checks (critical keys)
add_test("Null order_id", df["order_id"].isna().sum(), total)
add_test("Null customer_id", df["customer_id"].isna().sum(), total)
add_test("Null product_id", df["product_id"].isna().sum(), total)

# 2) Duplicate key check for fact line key: (order_id, order_item_id)
dup_keys = df.duplicated(subset=["order_id", "order_item_id"]).sum()
add_test("Duplicate (order_id, order_item_id)", dup_keys, total)

# 3) Validity checks
add_test("Price < 0", (df["price"] < 0).sum(), total)
add_test("Freight < 0", (df["freight_value"] < 0).sum(), total)
add_test("Total_paid < 0", (df["total_paid"] < 0).sum(), total)

# 4) Date logic checks (only when dates exist)
date_logic_fail = ((df["delivered_date"].notna()) & (df["order_date"].notna()) & (df["delivered_date"] < df["order_date"])).sum()
add_test("Delivered date earlier than order date", date_logic_fail, total)

# 5) Delivery days sanity
delivery_days_neg = (df["delivery_days"].notna() & (df["delivery_days"] < 0)).sum()
add_test("Negative delivery_days", delivery_days_neg, total)

quality_report = pd.DataFrame(tests)
quality_report

,test_name,failed_rows,total_rows,failed_pct,status
0,Null order_id,0,112650,0.0,PASS
1,Null customer_id,0,112650,0.0,PASS
2,Null product_id,0,112650,0.0,PASS
3,"Duplicate (order_id, order_item_id)",0,112650,0.0,PASS
4,Price < 0,0,112650,0.0,PASS
5,Freight < 0,0,112650,0.0,PASS
6,Total_paid < 0,0,112650,0.0,PASS
7,Delivered date earlier than order date,0,112650,0.0,PASS
8,Negative delivery_days,0,112650,0.0,PASS


In [33]:
output_path = r"C:\Users\vedas\Downloads\enterprise-analytics-platform\data\curated\data_quality_report.csv"
quality_report.to_csv(output_path, index=False)

print("✅ Data quality report saved to:", output_path)

✅ Data quality report saved to: C:\Users\vedas\Downloads\enterprise-analytics-platform\data\curated\data_quality_report.csv


In [34]:
conn.close()
print("✅ DB connection closed")

✅ DB connection closed


## 9. KPI Framework

Monthly KPIs were calculated to track business performance across
finance and operations. These KPIs are commonly used to monitor
order volume, revenue trends, and delivery performance.


In [27]:
import sqlite3
import pandas as pd

db_path = r"C:\Users\vedas\Downloads\enterprise-analytics-platform\data\raw\enterprise_analytics.db"
sql_path = r"C:\Users\vedas\Downloads\enterprise-analytics-platform\sql\05_kpi_monthly.sql"

conn = sqlite3.connect(db_path)

with open(sql_path, "r", encoding="utf-8") as f:
    query = f.read()

kpi_monthly = pd.read_sql_query(query, conn)
kpi_monthly.head()


,order_month,total_orders,gross_revenue,avg_order_value,avg_delivery_days,on_time_delivery_rate
0,2016-09,1,143.46,143.46,55.0,0.0000
1,2016-10,265,46490.66,175.44,19.6,0.9936
2,2016-12,1,19.62,19.62,5.0,1.0000
3,2017-01,750,127482.37,169.98,12.8,0.9737
4,2017-02,1653,271239.32,164.09,13.3,0.9688


In [28]:
output_path = r"C:\Users\vedas\Downloads\enterprise-analytics-platform\data\curated\monthly_kpis.csv"
kpi_monthly.to_csv(output_path, index=False)

print("✅ Monthly KPI file saved to:", output_path)


✅ Monthly KPI file saved to: C:\Users\vedas\Downloads\enterprise-analytics-platform\data\curated\monthly_kpis.csv
